# Fibre packer jobs

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*


This is exaggerated in terms of repetitions :-)

In [1]:
import os
import fibre_packer as fp

In [ ]:
# Parameters 
domain_radius = 70  # Domain radius
fibre_radius_mean = 2  # Mean fibre radius
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius
number_slices = 20 # Number of slices to generate
z_multiplier = 10 # Multiplier for z-coordinates, will be used later
iters_slice, repetitions_slice, change_slice = 200, 10, 3
iters_config, repetitions_config, change_config = 200, 30, 9
foldername = 'batch_second'

In [ ]:
for fibre_volume_fraction in [30, 40, 50, 60, 70]:
    print(f'Fibre volume fraction: {fibre_volume_fraction}')
    fib = fp.from_fvf(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
    fib.initialize_start_slice()
    losses = fib.optimize_slice_heuristic('start', iters=iters_slice, repetitions=repetitions_slice, change_every=change_slice)
    for misalignment in ['very low', 'low', 'moderate', 'high', 'very high']:
        print(f'Misalignment: {misalignment}')
        fib.initialize_end_slice(misalignment)
        losses = fib.optimize_slice_heuristic('end', iters=iters_slice, repetitions=repetitions_slice, change_every=change_slice)
        fib.interpolate_configuration(number_slices, z_multiplier)
        losses = fib.optimize_configuration_heuristic(iters=iters_config, repetitions=repetitions_config, change_every=change_config)
        fib.save_result(f'{foldername}/{misalignment}_{fibre_volume_fraction}.txt')
        fib.save_mesh(f'{foldername}/{misalignment}_{fibre_volume_fraction}.obj', n=8, close_ends=True)
        before = fib.get_fvp()
        fib.set_radii(fib.fix_radii())
        fib.save_result(f'{foldername}/{misalignment}_{fibre_volume_fraction}_fixed.txt')
        fib.save_mesh(f'{foldername}/{misalignment}_{fibre_volume_fraction}_fixed.obj', n=8, close_ends=True)
        after = fib.get_fvp()
        with open(f'{foldername}/_log.txt', 'a') as f:
            f.write(f'Asked: fvf {fibre_volume_fraction} {misalignment}, got {after:.5}/{before:.5}\n')